# 1. Import libraries & define global variables

In [56]:
import sys
print(sys.executable)

%pip install pyarrow xlsxwriter

c:\Users\bakan\AppData\Local\Programs\Python\Python313\python.exe
Note: you may need to restart the kernel to use updated packages.


In [57]:
from dataclasses import dataclass, field
from joblib import delayed, Parallel
from time import perf_counter
from typing import Self
from pathlib import Path
from tqdm import tqdm


import re
import numpy as np
import polars as pl
import pandas as pd

# CONFIG
XLSX_PATH = Path("simulations_data.xlsx")  # your workbook
SMOKER_SHEET_CANDIDATES    = ["Smoker", "Smoker ", "smoker"]
NONSMOKER_SHEET_CANDIDATES = ["Non-Smoker", "NonSmoker", "Non-smoker", "Non-Smoker ", "non-smoker"]

N_MONTHS = 60 # the first 5 years
MU_NONSMOKER, MU_SMOKER = 0.03, 0.09

N_POLICYHOLDERS = 1_000
BENEFIT = 100_000
EXPENSE_PER_ALIVE = 100

PREMIUM_NONSMOKER = 267.2840355         
PREMIUM_SMOKER = 1240.774524   

ANNUAL_RATE = 0.05             
HORIZON_M = 60                  # months to accumulate, i.e. first 5 years)
r = (1 + ANNUAL_RATE) ** (1 / 12) - 1 # monthly interest
discount_factor = 1/(1+r) 

# 2. Create helper classes & functions

In [58]:
def _normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Lowercase, strip, and replace spaces with underscores for column names."""
    mapping = {c: re.sub(r"\s+", "_", str(c).strip().lower()) for c in df.columns}
    return df.rename(columns=mapping)


def _detect_columns(df_norm: pd.DataFrame) -> dict:
    """
    Detect required columns in a sheet that already has:
    Month, Scenario, Alive at Start, Deaths
    """
    candidates = {
        "month":  ["month", "month_t", "mon"],
        "scen":   ["scenario_#", "scenario", "scenario_id", "sim", "sim_id"],
        "alive":  ["#_alive_at_start", "alive_at_start", "n_alive_start", "#_alive_start"],
        "deaths": ["#_deaths", "deaths", "num_deaths"],
    }
    out = {}
    for key, options in candidates.items():
        for name in options:
            if name in df_norm.columns:
                out[key] = name
                break
        if key not in out:
            raise ValueError(
                f"Missing column for '{key}'. Expected {options}. "
                f"Found columns: {list(df_norm.columns)}"
            )
    return {
        "month_col": out["month"],
        "scen_col": out["scen"],
        "alive_start_col": out["alive"],
        "deaths_col": out["deaths"],
    }

def _read_group_sheet(xlsx_path: Path, name_candidates: list[str]) -> tuple[pd.DataFrame, dict, str]:
    """Read a sheet by trying candidate names; return normalized DF, colmap, and actual sheet name."""
    xls = pd.ExcelFile(xlsx_path)
    sheet_to_use = None
    for nm in xls.sheet_names:
        if any(nm.strip().lower() == c.strip().lower() for c in name_candidates):
            sheet_to_use = nm
            break
    if sheet_to_use is None:
        raise ValueError(
            f"None of expected sheet names found {name_candidates}. "
            f"Workbook sheets: {xls.sheet_names}"
        )
    df = pd.read_excel(xlsx_path, sheet_name=sheet_to_use)
    df_norm = _normalize_columns(df)
    colmap = _detect_columns(df_norm)
    return df_norm, colmap, sheet_to_use


def _build_scenario_dict(df_norm: pd.DataFrame, colmap: dict) -> dict[int, dict[str, np.ndarray]]:
    """
    Build a dict keyed by scenario number:
      scen_dict[scenario] = {
          "month": np.array[int] (0..M),
          "alive_start": np.array[float],
          "deaths": np.array[float]
      }
    """
    scen_key = colmap["scen_col"]
    month_key = colmap["month_col"]
    alive_key = colmap["alive_start_col"]
    deaths_key = colmap["deaths_col"]

    d: dict[int, dict[str, np.ndarray]] = {}
    for scen, g in df_norm.groupby(scen_key, sort=True):
        g2 = g.sort_values(by=[month_key])
        d[int(scen)] = {
            "month": g2[month_key].to_numpy(dtype=int),
            "alive_start": g2[alive_key].to_numpy(dtype=float),
            "deaths": g2[deaths_key].to_numpy(dtype=float),
        }
    return d

def discount_benefit_column(df, scen_col, group_col, month_col, benefit_col, r_month):
    """
    Discounts benefit-only cashflows (Loss($) = benefits) using DF(t) = (1/(1+r_month))^t.
    Adds DF, PV_Benefit, PV_Cum.
    """
    df = df.copy().sort_values([scen_col, group_col, month_col])
    
    # ensure Month starts at 0
    t = df[month_col].to_numpy(float)
    discount_factor = 1.0 / (1.0 + r_month)
    df["DF"] = discount_factor ** t
    
    # PV of benefit
    df["PV_Benefit"] = df[benefit_col].astype(float) * df["DF"]
    
    # running sum = reserve path (present value)
    df["PV_Cum"] = 0.0
    for (scen, grp), g in df.groupby([scen_col, group_col], sort=False):
        idx = g.index
        df.loc[idx, "PV_Cum"] = g["PV_Benefit"].cumsum().to_numpy()
    
    return df

def _pv_from_arrays(months, alive_start, deaths, *,
                    premium, benefit, expense, r, horizon_m):
    """
    Present value version using discount_factor = 1/(1+r_month).
    DF(t) = discount_factor ** t.
    """
    M = int(horizon_m)

    alive = np.zeros(M+1); death = np.zeros(M+1)
    for m, a, dth in zip(months, alive_start, deaths):
        if 0 <= m <= M:
            alive[m] = a; death[m] = dth

    premium_in  = alive * premium
    expense_out = alive * expense
    benefit_out = death * benefit
    net_cf = expense_out + benefit_out - premium_in  # loss-positive

    discount_factor = 1.0 / (1.0 + r)          # r is MONTHLY
    t = np.arange(M+1, dtype=float)
    DF = discount_factor ** t                   # DF(t)

    pv_net = net_cf * DF                        # PV of loss in month t
    pv_cum = np.cumsum(pv_net)                  # PV up to month t

    return {
        "premium_in":  premium_in,
        "expense_out": expense_out,
        "benefit_out": benefit_out,
        "net_cf":      net_cf,
        "DF":          DF,
        "pv_net":      pv_net,
        "pv_cum":      pv_cum,
    }


def prospective_predicted_reserve_arrays(benefit_cf: np.ndarray,
                                         expense_cf: np.ndarray,
                                         premium_cf: np.ndarray,
                                         DF0: np.ndarray):
    """
    Returns:
      rem_pv0_ben/exp/prem[t]  : remaining PV at time 0 of future flows from t..M
      reserve_t[t]             : (benefit + expense - premium) valued at time t
    Conventions: benefits & expenses are outflows (+), premiums are inflows (+).
    """
    pv0_b = benefit_cf * DF0
    pv0_e = expense_cf * DF0
    pv0_p = premium_cf * DF0

    rsum = lambda x: np.flip(np.cumsum(np.flip(x)))
    rem_pv0_b = rsum(pv0_b)
    rem_pv0_e = rsum(pv0_e)
    rem_pv0_p = rsum(pv0_p)

    eps = 1e-15
    reserve_t = (rem_pv0_b + rem_pv0_e - rem_pv0_p) / np.maximum(DF0, eps)
    return (rem_pv0_b, rem_pv0_e, rem_pv0_p), reserve_t


# 3. Main FLow

## 3.1. Calculation

In [59]:
# # --- build rows (Smoker / Non-Smoker / Portfolio) with Predicted Reserve --- #
# rows = []
# scenarios = sorted(set(smoker_dict.keys()) | set(nonsmoker_dict.keys()))

# for scen in scenarios:
#     # smoker arrays (defaults if missing)
#     s = smoker_dict.get(
#         scen, {"month": np.arange(HORIZON_M + 1),
#                "alive_start": np.zeros(HORIZON_M + 1),
#                "deaths": np.zeros(HORIZON_M + 1)}
#     )
#     s_res = _pv_from_arrays(
#         s["month"], s["alive_start"], s["deaths"],
#         premium=PREMIUM_SMOKER, benefit=BENEFIT, expense=EXPENSE_PER_ALIVE,
#         r=r, horizon_m=HORIZON_M
#     )

#     # non-smoker arrays (defaults if missing)
#     n = nonsmoker_dict.get(
#         scen, {"month": np.arange(HORIZON_M + 1),
#                "alive_start": np.zeros(HORIZON_M + 1),
#                "deaths": np.zeros(HORIZON_M + 1)}
#     )
#     n_res = _pv_from_arrays(
#         n["month"], n["alive_start"], n["deaths"],
#         premium=PREMIUM_NONSMOKER, benefit=BENEFIT, expense=EXPENSE_PER_ALIVE,
#         r=r, horizon_m=HORIZON_M
#     )

#     # --- prospective (predicted) reserves from raw CFs and DF0 = v^t --- #
#     # Smoker
#     (_, _, _), s_reserve_t = prospective_predicted_reserve_arrays(
#         benefit_cf=s_res["benefit_out"],   # raw month CFs (not PV’d)
#         expense_cf=s_res["expense_out"],
#         premium_cf=s_res["premium_in"],
#         DF0=s_res["DF"]                    # DF0[t] = v^t
#     )
#     # Non-Smoker
#     (_, _, _), n_reserve_t = prospective_predicted_reserve_arrays(
#         benefit_cf=n_res["benefit_out"],
#         expense_cf=n_res["expense_out"],
#         premium_cf=n_res["premium_in"],
#         DF0=n_res["DF"]
#     )

#     # assemble rows
#     for m in range(HORIZON_M + 1):
#         # smoker row
#         rows.append({
#             "Scenario #": scen,
#             "Month": m,
#             "Group": "Smoker",
#             "# Alive at Start": s["alive_start"][m] if m < len(s["alive_start"]) else 0.0,
#             "# Deaths":        s["deaths"][m]      if m < len(s["deaths"])      else 0.0,
#             "Premium In":      s_res["premium_in"][m],
#             "Expense Out":     s_res["expense_out"][m],
#             "Benefit Out":     s_res["benefit_out"][m],
#             "Net CF":          s_res["net_cf"][m],
#             "PV Net":          s_res["pv_net"][m],
#             "PV Cum":          s_res["pv_cum"][m],
#             "Predicted Reserve": s_reserve_t[m],         # prospective
#         })

#         # non-smoker row  (FIX: use n_res for PV columns)
#         rows.append({
#             "Scenario #": scen,
#             "Month": m,
#             "Group": "Non-Smoker",
#             "# Alive at Start": n["alive_start"][m] if m < len(n["alive_start"]) else 0.0,
#             "# Deaths":        n["deaths"][m]      if m < len(n["deaths"])      else 0.0,
#             "Premium In":      n_res["premium_in"][m],
#             "Expense Out":     n_res["expense_out"][m],
#             "Benefit Out":     n_res["benefit_out"][m],
#             "Net CF":          n_res["net_cf"][m],
#             "PV Net":          n_res["pv_net"][m],       # <-- was s_res (bug), now n_res
#             "PV Cum":          n_res["pv_cum"][m],       # <-- was s_res (bug), now n_res
#             "Predicted Reserve": n_reserve_t[m],
#         })

#         # portfolio row (simple totals)
#         rows.append({
#             "Scenario #": scen,
#             "Month": m,
#             "Group": "Portfolio",
#             "# Alive at Start": (s["alive_start"][m] if m < len(s["alive_start"]) else 0.0) +
#                                 (n["alive_start"][m] if m < len(n["alive_start"]) else 0.0),
#             "# Deaths":        (s["deaths"][m]      if m < len(s["deaths"])      else 0.0) +
#                                 (n["deaths"][m]      if m < len(n["deaths"])      else 0.0),
#             "Premium In":      s_res["premium_in"][m]  + n_res["premium_in"][m],
#             "Expense Out":     s_res["expense_out"][m] + n_res["expense_out"][m],
#             "Benefit Out":     s_res["benefit_out"][m] + n_res["benefit_out"][m],
#             "Net CF":          s_res["net_cf"][m]      + n_res["net_cf"][m],
#             "PV Net":          s_res["pv_net"][m]      + n_res["pv_net"][m],
#             "PV Cum":          s_res["pv_cum"][m]      + n_res["pv_cum"][m],
#             "Predicted Reserve": s_reserve_t[m] + n_reserve_t[m],
#         })

# # Long table
# result = pd.DataFrame(rows).sort_values(["Scenario #", "Group", "Month"], kind="stable")

# # --- per-policy reserves and template-like portfolio reconstruction --- #
# eps = 1e-12
# result["Reserve_per_policy"] = result["Predicted Reserve"] / np.maximum(result["# Alive at Start"], eps)

# # Pivot per scenario/month so we can compute
# # PortfolioReserve_likeTemplate = S_alive*S_per + N_alive*N_per
# wide = (result[["Scenario #","Month","Group","# Alive at Start","Reserve_per_policy"]]
#         .pivot(index=["Scenario #","Month"], columns="Group",
#                values=["# Alive at Start","Reserve_per_policy"]))

# # Adjust group keys if your Group strings differ
# S_alive = wide["# Alive at Start"]["Smoker"].fillna(0.0)
# N_alive = wide["# Alive at Start"]["Non-Smoker"].fillna(0.0)
# S_per   = wide["Reserve_per_policy"]["Smoker"].fillna(0.0)
# N_per   = wide["Reserve_per_policy"]["Non-Smoker"].fillna(0.0)

# port_like = (S_alive * S_per + N_alive * N_per).rename("PortfolioReserve_likeTemplate").reset_index()

# # For convenience, also attach this back to the long table (Portfolio rows)
# result = result.merge(port_like, on=["Scenario #","Month"], how="left")

# # --- summaries across scenarios (mean & 95% bands) --- #
# def q(x, p): return np.percentile(x, p)

# summary = (
#     result.groupby(["Group","Month"], as_index=False)
#           .agg(
#               PV_Cum_mean = ("PV Cum", "mean"),
#               PV_Cum_p2_5 = ("PV Cum", lambda x: q(x, 2.5)),
#               PV_Cum_p97_5= ("PV Cum", lambda x: q(x, 97.5)),

#               PredRes_mean  = ("Predicted Reserve", "mean"),
#               PredRes_p2_5  = ("Predicted Reserve", lambda x: q(x, 2.5)),
#               PredRes_p97_5 = ("Predicted Reserve", lambda x: q(x, 97.5)),
#           )
# )

# # Add a separate summary for the template-like portfolio reserve (Portfolio rows only)
# port_summary = (
#     result[result["Group"] == "Portfolio"]
#       .groupby("Month", as_index=False)
#       .agg(TemplateLike_mean  = ("PortfolioReserve_likeTemplate", "mean"),
#            TemplateLike_p2_5  = ("PortfolioReserve_likeTemplate", lambda x: q(x, 2.5)),
#            TemplateLike_p97_5 = ("PortfolioReserve_likeTemplate", lambda x: q(x, 97.5)))
# )

# out_path = Path("reserves_from_two_sheets.xlsx")
# with pd.ExcelWriter(out_path, engine="xlsxwriter") as writer:
#     pd.DataFrame({
#         "BENEFIT": [BENEFIT],
#         "EXPENSE_PER_ALIVE": [EXPENSE_PER_ALIVE],
#         "PREMIUM_SMOKER": [PREMIUM_SMOKER],
#         "PREMIUM_NONSMOKER": [PREMIUM_NONSMOKER],
#         "ANNUAL_RATE": [ANNUAL_RATE],
#         "MONTHLY_r": [r],
#         "HORIZON_M": [HORIZON_M],
#     }).to_excel(writer, sheet_name="Inputs", index=False)

#     result.to_excel(writer, sheet_name="Paths_Long", index=False)
#     summary.to_excel(writer, sheet_name="Summary_Mean95", index=False)
#     port_summary.to_excel(writer, sheet_name="Portfolio_TemplateLike", index=False)

# print(f"Done. Wrote: {out_path.resolve()}")


In [ ]:
assert XLSX_PATH.exists(), f"Workbook not found: {XLSX_PATH}"

# Read two sheets independently
smoker_df_norm, smoker_cols, smoker_sheet = _read_group_sheet(
    XLSX_PATH, SMOKER_SHEET_CANDIDATES
)
nonsmoker_df_norm, nonsmoker_cols, nonsmoker_sheet = _read_group_sheet(
    XLSX_PATH, NONSMOKER_SHEET_CANDIDATES
)

# Build {scenario -> arrays}
smoker_dict = _build_scenario_dict(smoker_df_norm, smoker_cols)
nonsmoker_dict = _build_scenario_dict(nonsmoker_df_norm, nonsmoker_cols)

# Iterate scenarios, then months, compute results
scenarios = sorted(set(smoker_dict.keys()) | set(nonsmoker_dict.keys()))
rows = []

for scen in scenarios:
    # smoker arrays
    s = smoker_dict.get(
        scen,
        {
            "month": np.arange(HORIZON_M + 1),
            "alive_start": np.zeros(HORIZON_M + 1),
            "deaths": np.zeros(HORIZON_M + 1),
        },
    )
    s_res = _pv_from_arrays(
        s["month"], s["alive_start"], s["deaths"], 
        premium=PREMIUM_SMOKER, benefit=BENEFIT, expense=EXPENSE_PER_ALIVE,
        r=r, horizon_m=HORIZON_M
    )

    # non-smoker arrays
    n = nonsmoker_dict.get(
        scen,
        {
            "month": np.arange(HORIZON_M + 1),
            "alive_start": np.zeros(HORIZON_M + 1),
            "deaths": np.zeros(HORIZON_M + 1),
        },
    )
    n_res = _pv_from_arrays(
        n["month"], n["alive_start"], n["deaths"],
        premium=PREMIUM_NONSMOKER, benefit=BENEFIT, expense=EXPENSE_PER_ALIVE, 
        r=r, horizon_m=HORIZON_M
    )

    # smoker prospective reserve
    (_, _, _), s_reserve_t = prospective_predicted_reserve_arrays(
        benefit_cf=s_res["benefit_out"],
        expense_cf=s_res["expense_out"],
        premium_cf=s_res["premium_in"],
        DF0=s_res["DF"]
    )

    # non-smoker prospective reserve
    (_, _, _), n_reserve_t = prospective_predicted_reserve_arrays(
        benefit_cf=n_res["benefit_out"],
        expense_cf=n_res["expense_out"],
        premium_cf=n_res["premium_in"],
        DF0=n_res["DF"]
    )

    # assemble rows for each month (Smoker, Non-Smoker, Portfolio)
    for m in range(HORIZON_M + 1):
        # smoker
        rows.append({
            "Scenario #": scen,
            "Month": m,
            "Group": "Smoker",
            "# Alive at Start": s["alive_start"][m] if m < len(s["alive_start"]) else 0.0,
            "# Deaths":        s["deaths"][m]      if m < len(s["deaths"])      else 0.0,
            "Premium In":      s_res["premium_in"][m],
            "Expense Out":     s_res["expense_out"][m],
            "Benefit Out":     s_res["benefit_out"][m],
            "Net CF":          s_res["net_cf"][m],
            "PV Net": s_res["pv_net"][m],
            "PV Cum": s_res["pv_cum"][m],
            "Predicted Reserve": s_reserve_t[m],
        })
        # non-smoker
        rows.append({
            "Scenario #": scen,
            "Month": m,
            "Group": "Non-Smoker",
            "# Alive at Start": n["alive_start"][m] if m < len(n["alive_start"]) else 0.0,
            "# Deaths":        n["deaths"][m]      if m < len(n["deaths"])      else 0.0,
            "Premium In":      n_res["premium_in"][m],
            "Expense Out":     n_res["expense_out"][m],
            "Benefit Out":     n_res["benefit_out"][m],
            "Net CF":          n_res["net_cf"][m],
            "PV Net": n_res["pv_net"][m],
            "PV Cum": n_res["pv_cum"][m],
            "Predicted Reserve": n_reserve_t[m],
        })
        # portfolio (sum)
        rows.append({
            "Scenario #": scen,
            "Month": m,
            "Group": "Portfolio",
            "# Alive at Start": (s["alive_start"][m] if m < len(s["alive_start"]) else 0.0) +
                                (n["alive_start"][m] if m < len(n["alive_start"]) else 0.0),
            "# Deaths":        (s["deaths"][m]      if m < len(s["deaths"])      else 0.0) +
                                (n["deaths"][m]      if m < len(n["deaths"])      else 0.0),
            "Premium In":      s_res["premium_in"][m]  + n_res["premium_in"][m],
            "Expense Out":     s_res["expense_out"][m] + n_res["expense_out"][m],
            "Benefit Out":     s_res["benefit_out"][m] + n_res["benefit_out"][m],
            "Net CF":          s_res["net_cf"][m]      + n_res["net_cf"][m],
            "PV Net":          s_res["pv_net"][m]      + n_res["pv_net"][m],
            "PV Cum":          s_res["pv_cum"][m]      + n_res["pv_cum"][m],
            "Predicted Reserve": s_reserve_t[m] + n_reserve_t[m],
        })

result = pd.DataFrame(rows).sort_values(["Scenario #", "Group", "Month"], kind="stable")

# Summary across scenarios (mean & 95% band) per group/month
summary = (
    result.groupby(["Group", "Month"], as_index=False)
          .agg(
              PredRes_mean  = ("Predicted Reserve", "mean"),
              PredRes_p2_5  = ("Predicted Reserve", lambda x: np.percentile(x, 2.5)),
              PredRes_p97_5 = ("Predicted Reserve", lambda x: np.percentile(x, 97.5)),
          )
)

# Test
# 1) Portfolio should equal Smoker + Non-Smoker per scenario-month
chk = (result.pivot_table(index=["Scenario #","Month"],
                          columns="Group",
                          values="Predicted Reserve",
                          aggfunc="sum"))
if {"Smoker","Non-Smoker","Portfolio"}.issubset(chk.columns):
    delta = (chk["Portfolio"] - (chk["Smoker"] + chk["Non-Smoker"])).abs()
    print("Portfolio = S+N (abs diff) — max:", float(delta.max()), " mean:", float(delta.mean()))

# 2) Recompute the scenario-mean from Paths_Long and compare to Summary
recalc = (result.groupby(["Group","Month"], as_index=False)
                .agg(PredRes_mean_recalc=("Predicted Reserve","mean")))
merged = summary.merge(recalc, on=["Group","Month"], how="left")
merged["delta"] = merged["PredRes_mean"] - merged["PredRes_mean_recalc"]
print("Mean delta (should be ~0): max=", float(merged["delta"].abs().max()))


# Write Excel
out_path = Path("reserves_from_two_sheets.xlsx")
try:
    with pd.ExcelWriter(out_path, engine="xlsxwriter") as writer:
        pd.DataFrame({
            "BENEFIT": [BENEFIT],
            "EXPENSE_PER_ALIVE": [EXPENSE_PER_ALIVE],
            "PREMIUM_SMOKER": [PREMIUM_SMOKER],
            "PREMIUM_NONSMOKER": [PREMIUM_NONSMOKER],
            "ANNUAL_RATE": [ANNUAL_RATE],
            "MONTHLY_r": [r],
            "HORIZON_M": [HORIZON_M],
        }).to_excel(writer, sheet_name="Inputs", index=False)
        result.to_excel(writer, sheet_name="Paths_Long", index=False)
        summary.to_excel(writer, sheet_name="Summary_Mean95", index=False)
except ModuleNotFoundError:
    # Fallback engine if xlsxwriter isn't installed
    with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
        pd.DataFrame({
            "BENEFIT": [BENEFIT],
            "EXPENSE_PER_ALIVE": [EXPENSE_PER_ALIVE],
            "PREMIUM_SMOKER": [PREMIUM_SMOKER],
            "PREMIUM_NONSMOKER": [PREMIUM_NONSMOKER],
            "ANNUAL_RATE": [ANNUAL_RATE],
            "MONTHLY_r": [r],
            "HORIZON_M": [HORIZON_M],
        }).to_excel(writer, sheet_name="Inputs", index=False)
        result.to_excel(writer, sheet_name="Paths_Long", index=False)
        summary.to_excel(writer, sheet_name="Summary_Mean95", index=False)

print(f"Done. Wrote: {out_path.resolve()}")


Portfolio = S+N (abs diff) — max: 0.0  mean: 0.0
Mean delta (should be ~0): max= 0.0
Done. Wrote: C:\Users\bakan\Desktop\New folder\Master's Stuff\refresher\reserves_from_two_sheets.xlsx
